# MLOPs Zoomcamp 2023 / 01-homework
By Valerie G., May 2023

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
def get_samples(df_cp, dataset_type, dv=DictVectorizer()):
    df = df_cp.copy()  # to avoid warning about changing original df
    print('dataset_type:', dataset_type)
    num_columns = len(df.columns)  # for homework answer
    print('num_columns:', num_columns)
    print('num_rows:', len(df))
    
    # calc duration and add to df
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)  # convert to min
    
    # for homework answers
    if dataset_type == 'train':
        duration_std = df.duration.std()
        percentage_not_outliers = ((df.duration >= 1) & (df.duration <= 60)).mean() * 100

    # remove outliers
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # pic feature columns and convert them to dictionaries
    location_colns = ['PULocationID', 'DOLocationID']
    df[location_colns] = df[location_colns].astype(str)
    input_dicts = df[location_colns].to_dict(orient='records')
    
    # convert dict to input features for the model
    if dataset_type == 'train':
        inputs = dv.fit_transform(input_dicts)
    else:
        inputs = dv.transform(input_dicts)
    print('inputs.shape', inputs.shape)
        
    # pic target column and get its values
    target_coln = 'duration'
    targets = df[target_coln].values
    print('targets.shape', targets.shape)
    
    # print homework answers
    if dataset_type == 'train': 
        print('\nQ1:', num_columns)
        print('Q2:', duration_std)
        print('Q3:', percentage_not_outliers)
        matrix_dim = len(inputs.shape)
        print('Q4:', matrix_dim, '\n')
    
    return inputs, targets, dv


def evaluate_model(model, inputs, targets):
    preds = model.predict(inputs)  # predict
    rmse = mean_squared_error(targets, preds, squared=False)  # evaluate
    return rmse

In [3]:
# read data
df_train = pd.read_parquet('./yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('./yellow_tripdata_2022-02.parquet')

# show data
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [4]:
# load dataset for model
X_train, y_train, dv = get_samples(df_train, dataset_type='train')
X_val, y_val, _ = get_samples(df_val, dataset_type='val', dv=dv)

dataset_type: train
num_columns: 19
num_rows: 2463931
inputs.shape (2421440, 515)
targets.shape (2421440,)

Q1: 19
Q2: 46.44530513776802
Q3: 98.27547930522405
Q4: 2 

dataset_type: val
num_columns: 19
num_rows: 2979431
inputs.shape (2918187, 515)
targets.shape (2918187,)


Output:
```
dataset_type: train
num_columns: 19
num_rows: 2463931
inputs.shape (2421440, 515)
targets.shape (2421440,)

Q1: 19
Q2: 46.44530513776499
Q3: 98.27547930522405
Q4: 2 

dataset_type: val
num_columns: 19
num_rows: 2979431
inputs.shape (2918187, 515)
targets.shape (2918187,)
```

In [5]:
# train model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [6]:
# evaluate
rmse_train = evaluate_model(model, X_train, y_train)  # on train set
print('Q5:', rmse_train)
rmse_val = evaluate_model(model, X_val, y_val)  # on validation set
print('Q6:', rmse_val)

Q5: 6.986191065500608
Q6: 7.786408015215065


Output:
```
Q5: 6.98619083458946
Q6: 7.78640785205543
```